# Overview

# Setup and Configuration

## Dev Settings

In [ ]:
# Ignore SSL just for dev
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Packages

In [ ]:
!pip install langchain
!pip install ibm-watson-machine-learning
!pip install wget
!pip install sentence-transformers
!pip install chromadb

## Import Utility Packages

In [ ]:
import os
import getpass

## Environment Variables and Keys

### watsonx URL

In [ ]:
try:
    wxa_url = os.environ["WXA_URL"]
except KeyError:
    wxa_url = getpass.getpass("Please enter your watsonx.ai URL domain (hit enter): ")

### watsonx API Key

In [ ]:
try:
    wxa_api_key = os.environ["WXA_API_KEY"]
except KeyError:
    wxa_api_key = getpass.getpass("Please enter your watsonx.ai API key (hit enter): ")

### watsonx Project ID

In [ ]:
try:
    wxa_project_id = os.environ["WXA_PROJECT_ID"]
except KeyError:
    wxa_project_id = getpass.getpass("Please enter your watsonx.ai Project ID (hit enter): ")

# Query

In [ ]:
query = "What did the president say about Ketanji Brown Jackson?"

# Language Model

In [ ]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

In [ ]:
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100
}

In [ ]:
model = Model(
    model_id=ModelTypes.GRANITE_13B_INSTRUCT,
    params=parameters,
    credentials={
        "url": wxa_url,
        "apikey": wxa_api_key
    },
    project_id=wxa_project_id
)

In [ ]:
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
granite_llm_ibm = WatsonxLLM(model=model)

# Basic Completion

In [ ]:
result = granite_llm_ibm(query)

In [ ]:
print(result)

# Data for Documents

## Download Data

In [ ]:
import wget

filename = "state_of_the_union.txt"
url = "https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt"

In [ ]:
if not os.path.isfile(filename):
    wget.download(url, out=filename)

## Load documents

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader(filename)
documents = loader.load()

## Split Documents

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

# Vector Store

In [ ]:
from langchain.vectorstores import Chroma
chroma_vectorstore = Chroma.from_documents(texts, embeddings)

# Similiarity Search

In [ ]:
texts_sim = chroma_vectorstore.similarity_search(query, k=3)

In [ ]:
print("Number of relevant texts: " + str(len(texts_sim)))

In [ ]:
print("First 100 characters of relevant texts.")
for i in range(len(texts_sim)):
        print("Text " + str(i) + ": " + str(texts_sim[i].page_content[0:100]))

# RAG Generation (Explicit Context Control)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(granite_llm_ibm, chain_type="stuff")
result = chain.run(input_documents=texts_sim, question=query)

In [ ]:
print("Query: " + query)
print("Result:" + result)

# RAG Q&A Chain

In [ ]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=granite_llm_ibm, chain_type="stuff", retriever=chroma_vectorstore.as_retriever())
result = qa.run(query)

In [ ]:
print("Query: " + query)
print("Result:" + result)